In [1]:
# reference for below code
#https://towardsdatascience.com/simple-and-fast-question-answering-system-using-huggingface-distilbert-single-batch-inference-bcf5a5749571

In [2]:
# install transformers api on the box
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 16.9MB/s 
     |████████████████████████████████| 1.1MB 31.7MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3bcf5a0c0db80589f61deafcea610d279778c3d46061a601a8698d084ac257e3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
# import the tokenizer and model
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
import tensorflow as tf

In [4]:
# initialize text
context = "Itachi is a member of the Uchiha Clan from the village of Konohagakure and the elder brother of Sasuke Uchiha. \
He grew up as the child prodigy of the Uchiha clan, establishing milestones for not only Sasuke, but also for future Uchiha to live up to. \
His early childhood was marred by war, the violence of which caused him to seek peace at any cost. \
Upon learning that his clan was planning a coup d'état to take control of Konoha, after his clan was accused of attacking the village with the \
Nine Tails, Itachi provides Konoha with intelligence on his clan's actions during his two years as a member of the ANBU. \
Over time, Itachi distances himself from his clan, as some clan members believed he murdered his best friend, Shisui Uchiha, \
to gain the Mangekyo Sharingan. In reality, as Shisui possesses the ability to subliminally manipulate others and shares Itachi's love \
for the village, he intended to use his gift on the Uchiha clan leader to stop the coup d'état. However, Shisui is mortally wounded by the \
Leaf High Council member Danzo Shimura, who then takes his eye to achieve his own ideal of peace. When Shisui entrusts Itachi with his \
remaining eye, Itachi covers up Shisui's death to make it appear it was his doing. Eventually, in return for Danzo's offer to spare his \
younger brother, Itachi slaughters his entire clan. After creating the misconception that he murdered their family in cold blood to \
give Sasuke the mindset to become strong enough to kill him once he is capable, Itachi leaves the village, although he warns Danzo not to \
harm Sasuke. Meeting up with Konoha's Third Hokage Hiruzen Sarutobi, who promises to do what he can to look after Sasuke, Itachi reveals his \
intentions to continue to protect the village from the shadows. Soon after perceiving them as a threat to Konoha, Itachi joins the Akatsuki to \
ensure they do not endanger his village. Itachi becomes good friends with his partner Kisame Hoshigaki, \
who expresses concern for Itachi's wellbeing."

# initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',return_token_type_ids = True)
# more about tokenizer
# https://huggingface.co/transformers/main_classes/tokenizer.html
# https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase

model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [5]:
print(context)

Itachi is a member of the Uchiha Clan from the village of Konohagakure and the elder brother of Sasuke Uchiha. He grew up as the child prodigy of the Uchiha clan, establishing milestones for not only Sasuke, but also for future Uchiha to live up to. His early childhood was marred by war, the violence of which caused him to seek peace at any cost. Upon learning that his clan was planning a coup d'état to take control of Konoha, after his clan was accused of attacking the village with the Nine Tails, Itachi provides Konoha with intelligence on his clan's actions during his two years as a member of the ANBU. Over time, Itachi distances himself from his clan, as some clan members believed he murdered his best friend, Shisui Uchiha, to gain the Mangekyo Sharingan. In reality, as Shisui possesses the ability to subliminally manipulate others and shares Itachi's love for the village, he intended to use his gift on the Uchiha clan leader to stop the coup d'état. However, Shisui is mortally wou

In [6]:
def get_answer(ques):

  # encode the context and question
  #encoding = tokenizer.encode_plus(ques,context)
  encoding = tokenizer(ques,context)

  # get input_ids and attention_mask
  input_ids, attention_mask = encoding['input_ids'], encoding['attention_mask']
  # more on above terms
  # https://huggingface.co/transformers/glossary.html

  # get the output from DistilBERT for above inputs
  start_scores, end_scores = model(torch.tensor([input_ids]),
                                   attention_mask=torch.tensor(attention_mask))

  # get the input ids corresponding to above scores
  answer_ids = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores) + 1]

  # get the tokens corresponding to above ids
  answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids,skip_special_tokens=True)

  # convert tokens to string
  answer_string = tokenizer.convert_tokens_to_string(answer_tokens)
  
  return answer_string

In [7]:
# prompt for question
q = input('Ask a question: ')
# get answer from context
a = get_answer(q)
print("\nAnswer: ",a)

Ask a question: Who is Itachi Uchiha?

Answer:  a member of the uchiha clan from the village of konohagakure and the elder brother


In [8]:
# prompt for question
q = input('Ask a question: ')
# get answer from context
a = get_answer(q)
print("\nAnswer: ",a)

Ask a question: What is the name of Third Hokage?

Answer:  hiruzen sarutobi


In [9]:
# prompt for question
q = input('Ask a question: ')
# get answer from context
a = get_answer(q)
print("\nAnswer: ",a)

Ask a question: how much time Itachi spent in ANBU?

Answer:  two years


In [10]:
# prompt for question
q = input('Ask a question: ')
# get answer from context
a = get_answer(q)
print("\nAnswer: ",a)

Ask a question: Which organization did Itachi join?

Answer:  akatsuki
